In [ ]:
# 2025-10-28 against production
from tqdm import tqdm
from django.core.exceptions import ObjectDoesNotExist
import requests
from dumper.utils import process_beacon
from normdata.utils import import_from_normdata
from AcdhArcheAssets.uri_norm_rules import get_normalized_uri

In [ ]:
beacon_url = "https://raw.githubusercontent.com/digicademy/sturm-exist-app/refs/heads/master/html/beacon/personen.txt"
domain = "sturm"

In [ ]:
r = requests.get(beacon_url)
lines = r.content.decode("utf-8").split("\n")
col, _ = Collection.objects.get_or_create(name="Der Sturm")

In [ ]:
for x in tqdm(lines, total=len(lines)):
    if "|" in x and not x.startswith("#"):
        gnd, beacon_uri = get_normalized_uri(x.split("|")[0]), x.split("|")[-1]
        gnd = f"https://d-nb.info/gnd/{gnd}"
        try:
            item = Uri.objects.get(uri=gnd)
            entity = item.entity
        except ObjectDoesNotExist:
            entity = import_from_normdata(gnd, "person")
        try:
            entity.collection.add(col)
        except AttributeError:
            print(gnd, beacon_uri)
            continue
        try:
            Uri.objects.get(uri=beacon_uri)
            continue
        except ObjectDoesNotExist:
            new_uri = Uri.objects.create(uri=beacon_uri, entity=entity)
            new_uri.domain = domain
            new_uri.save()